In [1]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt

import os
import pyopencl
import datetime
import sys

#Finally, import our simulator
from SWESimulators import KP07, DataOutput, PlotHelper, Common

#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')


In [2]:
reload(KP07)

<module 'SWESimulators.KP07' from 'SWESimulators\KP07.pyc'>

In [3]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

Using  GeForce 840M


In [4]:
#Clean up old simulator if any:
if 'sim' in globals():
    sim.cleanUp()
    
# Kurganov-Petrova 2007 paper
reload(KP07)
nx = 100
ny = 200

dx = 200.0
dy = 200.0

dt_scale = 0.9
g = 9.81

f = 0.00
r = 0.0

wind = Common.WindStressParams(type=99)
boundaryConditions = Common.BoundaryConditions()


bcSettings = 1
ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
waterHeight = 60

eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
addCentralBump(eta0, nx, ny, dx, dy, validDomain)
u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');

Hi = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C') * waterHeight;

#Initialize simulator
reload(KP07)
reload(Common)
sim = KP07.KP07(cl_ctx, \
                eta0, Hi, u0, v0, \
                nx, ny, \
                dx, dy, \
                g, f, r, \
                wind_stress=wind, \
                boundary_conditions=boundaryConditions,
                use_rk2=True, dt_scale=dt_scale)



SWESimulators\Common.py:57: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if (data != None):


In [6]:

#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                eta0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] )

T = 50
def animate(i):
    if (i>0):
        t = sim.step(10.1)
    else:
        t = 0.0
    eta1, u1, v1 = sim.download()

    plotter.plot(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] );
    fig.suptitle("KP07 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%1 == 0):
        print("{:03.0f}".format(100*i / T) + " % => t=" + str(t) 
            + "\tMax h: " + str(np.max(eta1)) 
            + "\tMax u: " + str(np.max(u1)) 
            + "\tMax v: " + str(np.max(v1)))
        #fig.savefig(imgdir + "/{:010.0f}_kp07.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

000 % => t=0.0	Max h: 1.0	Max u: 0.0	Max v: 0.0
000 % => t=0.0	Max h: 1.0	Max u: 0.0	Max v: 0.0
1.83956
1.83918
1.83853
1.83934
1.84116
0.902236
Computed 6 timesteps
002 % => t=10.1	Max h: 0.302871	Max u: 6.41804	Max v: 6.41804
1.84293
1.84378
1.84447
1.84505
1.8459
0.877872
Computed 6 timesteps
004 % => t=20.2	Max h: 0.148284	Max u: 4.30107	Max v: 4.30107
1.84712
1.84753
1.84798
1.84851
1.84884
0.860013
Computed 6 timesteps
006 % => t=30.3	Max h: 0.112243	Max u: 3.21052	Max v: 3.21052
1.84921
1.84949
1.84983
1.85004
1.8502
0.851216
Computed 6 timesteps
008 % => t=40.4	Max h: 0.0907982	Max u: 2.44075	Max v: 2.44077
1.85049
1.85065
1.85082
1.85095
1.85106
0.846039
Computed 6 timesteps
010 % => t=50.5	Max h: 0.077063	Max u: 1.96881	Max v: 1.96882
1.85125
1.85137
1.85146
1.85156
1.85166
0.842698
Computed 6 timesteps
012 % => t=60.6	Max h: 0.0675244	Max u: 1.66584	Max v: 1.66584
1.8518
1.85186
1.85194
1.85204
1.8521
0.840268
Computed 6 timesteps
014 % => t=70.7	Max h: 0.060323	Max u: 1.449